# Modèle AF (via PyAEZ)

On utilise la bibliothèque PyAEZ, open source de la FAO, qui fonctionne comme suit :
![title](images/pyaez_info.png)

On commence par inialiser la partie agro-climatique en utilisant la latitude minimale et la longitude maimale de la parecelle, supposée rectangulaire (possibilité de mettre un masque sinon). 

On utilise le module python soilgrids et ? pour réunir les informations sur la parcelle.

In [22]:
#imports de modules
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
try:
    from osgeo import gdal
except:
    import gdal
import sys
import pyaez.ClimateRegime as ClimateRegime
import pyaez.CropSimulation as CropSimulation
import pyaez.UtilitiesCalc as UtilitiesCalc
import pyaez.SoilConstraints as SoilConstraints
import pyaez.ClimaticConstraints as ClimaticConstraints

Le code est en version beta++ mais il marche, il reste à faire :
 - Connecter les API pour donner des valeurs réalistes
 - Modéliser plus précisément les arbres


In [23]:
# Modèle AF (via PyAEZ)

@dataclass
class TreeParameters:
    """Tree species parameters"""
    height: float  # meters
    crown_diameter: float  # meters
    leaf_area_index: float
    root_depth: float  # meters
    water_requirement: float  # mm/day
    shade_tolerance: float  # 0-1 scale
    nutrient_contribution: float  # kg/ha/year

def params_geo(lat_min, lat_max, freq):
    """Récupère les données geo obtenues via soilgrids selon la parcelle
    À implémenter avec l'API SoilGrids et ptet la nasa
    """
    #version non connectée (arbitraire)
    shape = (10, 10, 365 if freq == 'daily' else 12)
    return (
        np.ones(shape[:-1])*10,  #elevation
        np.ones(shape)*2,        #min_temp
        np.ones(shape)*25,       #max_temp
        np.ones(shape)*60,        #rel_humidity
        np.ones(shape)*1000,      #short_rad
        np.ones(shape)*2,        #wind_speed
        np.ones(shape)*5         #precipitation
    )

class AgroforestryModel:
    def __init__(self, lat_min, lat_max, freq='daily'):
        self.climate_regime = self._init_climate_regime(lat_min, lat_max, freq)
        self.crop_simulation = self._init_crop_simulation(lat_min, lat_max, freq)
        self.trees = []
        self.tree_density = 0
        self.tree_arrangement = None

    def _init_climate_regime(self, lat_min, lat_max, freq):
        """Initialise le modèle climatique"""
        clim_reg = ClimateRegime.ClimateRegime()
        elevation, min_temp, max_temp, rel_humidity, short_rad, wind_speed, precipitation = params_geo(lat_min, lat_max, freq)
        
        # Masques anti valeurs aberrantes
        rel_humidity[rel_humidity < 0] = 0
        short_rad[short_rad < 0] = 0
        wind_speed[wind_speed < 0] = 0
        # Initialisation du masque (si parcelle pas rectangulaire)
        mask = np.ones((10,10)) #pas de masque, à mettre à la bonne taille (ici 10*10) 
        mask_value = 0
        clim_reg.setStudyAreaMask(mask, mask_value)
        # Initialisation de la localisation de la parcelle
        clim_reg.setLocationTerrainData(lat_min, lat_max, elevation)
        if freq == 'daily':
            clim_reg.setDailyClimateData(min_temp, max_temp, precipitation, 
                                       short_rad, wind_speed, rel_humidity)
        else:
            clim_reg.setMonthlyClimateData(min_temp, max_temp, precipitation, 
                                          short_rad, wind_speed, rel_humidity)
        return clim_reg
    
    def _init_crop_simulation(self, lat_min, lat_max, freq):
        """Initialise le modèle de simulation des cultures"""
        crop_sim = CropSimulation.CropSimulation()
        elevation, min_temp, max_temp, rel_humidity, short_rad, wind_speed, precipitation = params_geo(lat_min, lat_max, freq)
        
        # Masques anti valeurs aberrantes
        rel_humidity[rel_humidity < 0] = 0
        short_rad[short_rad < 0] = 0
        wind_speed[wind_speed < 0] = 0
        # Initialisation du masque (si parcelle pas rectangulaire)
        mask = np.ones((10,10)) #pas de masque, à mettre à la bonne taille (ici 10*10) 
        mask_value = 0
        crop_sim.setStudyAreaMask(mask, mask_value)
        # Initialisation de la localisation de la parcelle
        crop_sim.setLocationTerrainData(lat_min, lat_max, elevation)
        if freq == 'daily':
            crop_sim.setDailyClimateData(min_temp, max_temp, precipitation, 
                                       short_rad, wind_speed, rel_humidity)
        else:
            crop_sim.setMonthlyClimateData(min_temp, max_temp, precipitation, 
                                          short_rad, wind_speed, rel_humidity)
        return crop_sim

    def add_tree_species(self, tree_params: TreeParameters):
        """Ajoute une espèce d'arbre au système agroforestier"""
        self.trees.append(tree_params)
    
    def set_tree_arrangement(self, density, pattern='alley'):
        """Définit la densité et le motif de plantation des arbres"""
        self.tree_density = density
        self.tree_arrangement = pattern
    
    def calculate_light_interception(self):
        """Calcule l'interception lumineuse par les arbres
        1 si pas d'abre et 0.3 au minimum (arbitraire) si toute la surface est prise par la canopée
        """
        if not self.trees:
            return 1.0
        
        total_canopy_area = sum(tree.crown_diameter ** 2 * np.pi / 4 * self.tree_density 
                               for tree in self.trees)
        total_area = self.
        light_transmission = 
        return light_transmission
    
    def calculate_water_competition(self):
        """Calcule la compétition pour l'eau entre arbres et cultures"""
        if not self.trees:
            return 1.0
        
        total_water_requirement = sum(tree.water_requirement * self.tree_density 
                                    for tree in self.trees)
        water_factor = 1 / (1 + 0.1 * total_water_requirement)
        return water_factor
    
    def calculate_nutrient_contribution(self):
        """Calcule l'apport en nutriments des arbres"""
        if not self.trees:
            return 0.0
        
        total_nutrient = sum(tree.nutrient_contribution * self.tree_density 
                            for tree in self.trees)
        return total_nutrient

    def setup_crop_parameters(self, LAI=2.3, HI=0.33, legume=0, adaptability=4, 
                            cycle_len=115, D1=0.3, D2=1, min_temp = -2, aLAI = 70,
                             bLAI = 200, aHI=120, bHI=180, min_cycle_len = 100, 
                              max_cycle_len = 200, plant_height = 1.5):
        """Configure les paramètres de culture
            LAI (float): Leaf Area Index
            HI (float): Harvest Index
            legume (binary, yes=1, no=0): Is the crop legume?
            adaptability (int): Crop adaptability clases (1-4)
            cycle_len (int): Length of crop cycle
            D1 (float): Rooting depth at the beginning of the crop cycle [m]
            D2 (float): Rooting depth after crop maturity [m]
            min_temp (int or float): minimum temperature requirement of the crop [deg C]
            aLAI (int or float): alpha LAI adjustment parameter
            bLAI (int or float): beta LAI adjustment parameter
            aHI (int): alpha coefficient for HI
            bHI (int): beta coefficient for HI
            min_cycle_len (int): minimum cycle length [days]
            max_cycle_len (int): maximum cycle length [days]
            plant_height (int or float): plant height [m]
            """
        self.crop_simulation.setCropParameters(LAI=LAI, HI=HI, legume=legume, 
            adaptability=adaptability, cycle_len=cycle_len, 
            D1=D1, D2=D2, min_temp=min_temp, aLAI=aLAI, bLAI=bLAI, aHI=aHI, bHI=bHI,
            min_cycle_len=min_cycle_len, max_cycle_len=max_cycle_len, plant_height=plant_height
        )        
        # Paramètres par défaut du cycle cultural
        self.crop_simulation.setCropCycleParameters(
            stage_per=[16, 26, 33, 25],
            kc=[0.3, 1.2, 0.5],
            kc_all=0.85,
            yloss_f=[0.4, 0.4, 0.9, 0.5],
            yloss_f_all=1.25
        )
        
        self.crop_simulation.setSoilWaterParameters(Sa=100, pc=0.5)

    def simulate_agroforestry_yield(self, lat_min, lat_max, freq, start_doy=1, end_doy=365):
        """Simule le rendement en tenant compte des interactions arbres-cultures"""
        # Configuration du screening climatique
        tclimate = self.climate_regime.getThermalClimate()
        self.crop_simulation.setThermalClimateScreening(tclimate, no_t_climate=[2, 5, 6])
        
        # Simulation de base
        self.crop_simulation.simulateCropCycle(start_doy, end_doy, step_doy=1, leap_year=False)
        base_yield_rain = self.crop_simulation.getEstimatedYieldRainfed()
        base_yield_irr = self.crop_simulation.getEstimatedYieldIrrigated()
        
        # Facteurs de modification agroforestière (modélisation très grossière)
        light_factor = self.calculate_light_interception()
        water_factor = self.calculate_water_competition()
        nutrient_bonus = self.calculate_nutrient_contribution()
        
        # Application des contraintes
        obj_utilities = UtilitiesCalc.UtilitiesCalc()
        #a changer on a besoin que de elevation, ptet mettre ces trucs dans l init de la classe dans un meta parametre params
        elevation, min_temp, max_temp, rel_humidity, short_rad, wind_speed, precipitation = params_geo(lat_min, lat_max, freq)
        clim_constraints = ClimaticConstraints.ClimaticConstraints(lat_min, lat_max, elevation)
        clim_reg = self.climate_regime
        lgpt0 = clim_reg.getThermalLGP0()
        lgpt5 = clim_reg.getThermalLGP5()
        lgpt10 = clim_reg.getThermalLGP10()
        lgp = clim_reg.getLGP(Sa=100)
        lgp_equv = clim_reg.getLGPEquivalent()
        
        # Modification des rendements avec effets agroforestiers
        modified_yield_rain = base_yield_rain * light_factor * water_factor * (1 + 0.1 * nutrient_bonus)
        modified_yield_irr = base_yield_irr * light_factor * (1 + 0.1 * nutrient_bonus)
        
        # Application des contraintes climatiques
        aez = self.crop_simulation
        yield_map_rain = aez.getEstimatedYieldRainfed()   # Avec pluie
        clim_adj_yield_rain = clim_constraints.applyClimaticConstraints(yield_map_rain, lgp, lgp_equv, lgpt10, omit_yld_0 = True)
        yield_map_irr = aez.getEstimatedYieldIrrigated()   # Avec irrigation
        clim_adj_yield_irr = clim_constraints.applyClimaticConstraints(yield_map_irr, lgp, lgp_equv, lgpt10, omit_yld_0 = True)
        
        return {
            'rainfed_yield': clim_adj_yield_rain,
            'irrigated_yield': clim_adj_yield_irr,
            'light_factor': light_factor,
            'water_factor': water_factor,
            'nutrient_contribution': nutrient_bonus,
            'lgp': lgp,
            'base_yield_rain': base_yield_rain,
            'base_yield_irr': base_yield_irr
        }

# Exemple d'utilisation
def run_example():
    # Initialisation du modèle
    lat_min=45.5
    lat_max=45.6
    model = AgroforestryModel(lat_min,lat_max)
    
    # Définition d'une espèce d'arbre (exemple avec un acacia)
    acacia = TreeParameters(
        height=8.0,
        crown_diameter=6.0,
        leaf_area_index=2.5,
        root_depth=2.0,
        water_requirement=50.0,
        shade_tolerance=0.7,
        nutrient_contribution=100.0
    )
    
    # Configuration du système agroforestier
    model.add_tree_species(acacia)
    model.set_tree_arrangement(density=100, pattern='alley')  # 100 arbres/ha
    
    # Configuration des paramètres de culture
    model.setup_crop_parameters()
    
    # Simulation
    results = model.simulate_agroforestry_yield(lat_min,lat_max,freq = 'daily')

    # Affichage des résultats
    print("Résultats de la simulation agroforestière:")
    print(f"Rendement pluvial: {results['rainfed_yield'] if results['rainfed_yield'] is not None else 'N/A'}")
    print(f"Rendement irrigué: {results['irrigated_yield'] if results['irrigated_yield'] is not None else 'N/A'}")
    print(f"Facteur d'ombrage: {results['light_factor']:.2f}")
    print(f"Facteur hydrique: {results['water_factor']:.2f}")
    print(f"Contribution en nutriments: {results['nutrient_contribution']:.2f} kg/ha/an")
    return results

if __name__ == "__main__":
    results = run_example()

Done %: 100.0
Simulations Completed !
Résultats de la simulation agroforestière:
Rendement pluvial: N/A
Rendement irrigué: N/A
Facteur d'ombrage: 0.00
Facteur hydrique: 0.00
Contribution en nutriments: 10000.00 kg/ha/an


In [26]:
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import math

@dataclass
class TreeParameters:
    """Enhanced tree species parameters based on Hi-sAFe model"""
    height: float  # meters
    crown_diameter: float  # meters
    leaf_area_index: float
    root_depth: float  # meters
    water_requirement: float  # mm/day
    shade_tolerance: float  # 0-1 scale
    nutrient_contribution: float  # kg/ha/year
    # New Hi-sAFe specific parameters
    crown_shape: str  # 'cone', 'ellipsoid', 'cylinder'
    crown_height: float  # meters
    trunk_diameter: float  # meters
    root_lateral_extent: float  # meters
    leaf_angle_distribution: float  # mean leaf inclination angle in degrees
    wood_density: float  # kg/m3
    specific_leaf_area: float  # m2/kg
    light_extinction_coeff: float  # Beer-Lambert law coefficient

class SoilLayer:
    """Representation of a soil layer with Hi-sAFe properties"""
    def __init__(self, depth: float, thickness: float):
        self.depth = depth
        self.thickness = thickness
        self.water_content = 0.0
        self.field_capacity = 0.3
        self.wilting_point = 0.1
        self.bulk_density = 1.4
        self.root_density = 0.0
        self.nutrient_content = {
            'N': 0.0,
            'P': 0.0,
            'K': 0.0
        }

class VoxelGrid:
    """3D spatial discretization for light and root competition"""
    def __init__(self, x_size: int, y_size: int, z_size: int, voxel_size: float):
        self.grid = np.zeros((x_size, y_size, z_size))
        self.voxel_size = voxel_size
        self.light_extinction = np.ones((x_size, y_size))
        self.root_occupation = np.zeros((x_size, y_size, z_size))
        self.water_content = np.zeros((x_size, y_size, z_size))

class AgroforestryModel:
    def __init__(self, lat_min, lat_max, freq='daily'):
        self.climate_regime = self._init_climate_regime(lat_min, lat_max, freq)
        self.crop_simulation = self._init_crop_simulation(lat_min, lat_max, freq)
        self.trees: List[TreeParameters] = []
        self.tree_positions: List[Tuple[float, float]] = []
        self.tree_density = 0
        self.tree_arrangement = None
        
        # Initialize 3D voxel grid for spatial modeling
        self.voxel_grid = VoxelGrid(50, 50, 20, 1.0)  # 1m voxel size
        self.soil_layers = [SoilLayer(d, 0.2) for d in np.arange(0, 2.0, 0.2)]
        
    def _init_climate_regime(self, lat_min, lat_max, freq):
        """Existing climate regime initialization"""
        # ... (keeping existing implementation)
        pass

    def _init_crop_simulation(self, lat_min, lat_max, freq):
        """Existing crop simulation initialization"""
        # ... (keeping existing implementation)
        pass

    def calculate_crown_volume(self, tree: TreeParameters) -> float:
        """Calculate tree crown volume based on shape"""
        if tree.crown_shape == 'cone':
            return (np.pi * (tree.crown_diameter/2)**2 * tree.crown_height) / 3
        elif tree.crown_shape == 'ellipsoid':
            return (4/3) * np.pi * (tree.crown_diameter/2)**2 * (tree.crown_height/2)
        else:  # cylinder
            return np.pi * (tree.crown_diameter/2)**2 * tree.crown_height

    def update_light_interception(self):
        """Update light interception using Hi-sAFe's 3D approach"""
        self.voxel_grid.light_extinction.fill(1.0)
        
        for tree, position in zip(self.trees, self.tree_positions):
            x, y = position
            radius = tree.crown_diameter / 2
            height = tree.height
            
            # Calculate shadow projection
            for i in range(self.voxel_grid.grid.shape[0]):
                for j in range(self.voxel_grid.grid.shape[1]):
                    dx = (i * self.voxel_grid.voxel_size) - x
                    dy = (j * self.voxel_grid.voxel_size) - y
                    distance = np.sqrt(dx**2 + dy**2)
                    
                    if distance <= radius:
                        # Apply Beer-Lambert law for light extinction
                        path_length = self.calculate_path_length(tree, distance)
                        extinction = np.exp(-tree.light_extinction_coeff * 
                                         tree.leaf_area_index * path_length)
                        self.voxel_grid.light_extinction[i, j] *= extinction

    def update_root_competition(self):
        """Update root competition using Hi-sAFe's 3D root model"""
        self.voxel_grid.root_occupation.fill(0.0)
        
        for tree, position in zip(self.trees, self.tree_positions):
            x, y = position
            
            # Calculate root distribution in 3D space
            for i in range(self.voxel_grid.grid.shape[0]):
                for j in range(self.voxel_grid.grid.shape[1]):
                    for k in range(self.voxel_grid.grid.shape[2]):
                        dx = (i * self.voxel_grid.voxel_size) - x
                        dy = (j * self.voxel_grid.voxel_size) - y
                        dz = k * self.voxel_grid.voxel_size
                        
                        # Use Hi-sAFe's root distribution model
                        distance = np.sqrt(dx**2 + dy**2)
                        if distance <= tree.root_lateral_extent and dz <= tree.root_depth:
                            root_density = self.calculate_root_density(tree, distance, dz)
                            self.voxel_grid.root_occupation[i, j, k] += root_density

    def calculate_root_density(self, tree: TreeParameters, distance: float, depth: float) -> float:
        """Calculate root density using Hi-sAFe's root distribution model"""
        # Simplified version of Hi-sAFe's root distribution
        lateral_factor = np.exp(-distance / tree.root_lateral_extent)
        depth_factor = np.exp(-depth / tree.root_depth)
        return lateral_factor * depth_factor

    def calculate_water_balance(self):
        """Calculate water balance using Hi-sAFe's water module"""
        for layer in self.soil_layers:
            # Update water content based on precipitation and extraction
            available_water = (layer.water_content - layer.wilting_point) * layer.thickness
            
            # Calculate water extraction by roots
            root_water_extraction = np.sum(self.voxel_grid.root_occupation) * available_water
            
            # Update layer water content
            layer.water_content = max(layer.wilting_point,
                                    layer.water_content - root_water_extraction)

    def simulate_agroforestry_yield(self, lat_min, lat_max, freq, start_doy=1, end_doy=365):
        """Enhanced yield simulation incorporating Hi-sAFe concepts"""
        # Update 3D model states
        self.update_light_interception()
        self.update_root_competition()
        self.calculate_water_balance()
        
        # Calculate integrated effects
        light_factor = np.mean(self.voxel_grid.light_extinction)
        water_factor = self.calculate_water_competition()
        nutrient_bonus = self.calculate_nutrient_contribution()
        
        # Base simulation
        base_yield_rain = self.crop_simulation.getEstimatedYieldRainfed()
        base_yield_irr = self.crop_simulation.getEstimatedYieldIrrigated()
        
        # Apply Hi-sAFe modifiers
        modified_yield_rain = (base_yield_rain * 
                             light_factor * 
                             water_factor * 
                             (1 + 0.1 * nutrient_bonus))
        modified_yield_irr = (base_yield_irr * 
                            light_factor * 
                            (1 + 0.1 * nutrient_bonus))
        
        return {
            'rainfed_yield': modified_yield_rain,
            'irrigated_yield': modified_yield_irr,
            'light_factor': light_factor,
            'water_factor': water_factor,
            'nutrient_contribution': nutrient_bonus,
            'light_distribution': self.voxel_grid.light_extinction.copy(),
            'root_competition': np.sum(self.voxel_grid.root_occupation, axis=2)
        }

# Example usage
def run_example():
    # Initialize model
    lat_min, lat_max = 45.5, 45.6
    model = AgroforestryModel(lat_min, lat_max)
    
    # Define tree parameters with Hi-sAFe specific attributes
    walnut = TreeParameters(
        height=12.0,
        crown_diameter=8.0,
        leaf_area_index=3.5,
        root_depth=3.0,
        water_requirement=60.0,
        shade_tolerance=0.6,
        nutrient_contribution=120.0,
        crown_shape='ellipsoid',
        crown_height=6.0,
        trunk_diameter=0.3,
        root_lateral_extent=5.0,
        leaf_angle_distribution=45.0,
        wood_density=650.0,
        specific_leaf_area=12.0,
        light_extinction_coeff=0.5
    )
    
    # Add trees with specific positions
    model.add_tree_species(walnut)
    model.tree_positions = [(10, 10), (20, 20), (30, 30)]  # Example positions
    model.tree_density = len(model.tree_positions) / (50 * 50)  # trees per m²
    
    # Run simulation
    results = model.simulate_agroforestry_yield(lat_min, lat_max, freq='daily')
    
    return results

if __name__ == "__main__":
    results = run_example()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\PyAEZ_iiasa\\data_input/climate/max_temp.npy'